In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.morphology import disk, opening, closing
import glob
import time
import pathlib

In [ ]:
rpi = 'h2r1'

# Read input image and resize them
path_to_folder = '../live_bees/' + rpi + '/random_images_of_50/*.jpg'
names_folder = '../live_bees/' + rpi + '/random_images_25/*.jpg'
folder_in = sorted(glob.glob(path_to_folder))
names = [pathlib.Path(x).stem for x in names_folder]
imgs = [cv2.imread(x) for x in folder_in]
n = 0

output_folder = '../a_processed_images/optical_flow/' + rpi + '/' 

In [ ]:
masks = []
times = []
frames = []
n = 0
for frame in frames:
    if n == 0:
        prev_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        mask = np.zeros_like(frames[0])
        mask[..., 1] = 255
        n += 1
        continue
    fig, ax = plt.subplots(1, 3, figsize=(15, 15))
    ax[0].imshow(frame)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
        
    # Calculates dense optical flow by Farneback method 
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, flow=None, 
                                        pyr_scale=0.5, # PyrScale=0.5 means a classical pyramid, where each next layer is twice smaller than the previous one. default 0.5.
                                        levels=3,      # Number of pyramid layers including the initial image. Levels=1 means that no extra layers are created and only the original images are used. default 5.
                                        winsize=15,    # Averaging window size. Larger values increase the algorithm robustness to image noise and give more chances for fast motion detection, but yield more blurred motion field. default 13.
                                        iterations=3,  # Number of iterations the algorithm does at each pyramid level. default 10.
                                        poly_n=5,      # Size of the pixel neighborhood used to find polynomial expansion in each pixel. Larger values mean that the image will be approximated with smoother surfaces, yielding more robust algorithm and more blurred motion field. Typically, PolyN is 5 or 7. default 5.
                                        poly_sigma=1.2,# Standard deviation of the Gaussian that is used to smooth derivatives used as a basis for the polynomial expansion. For PolyN=5, you can set PolySigma = 1.1. For PolyN=7, a good value would be PolySigma = 1.5. default 1.1.
                                        flags=0)
        
    # Computes the magnitude and angle of the 2D vectors 
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1]) 
        
    # Sets image hue according to the optical flow  
    # direction 
    mask[..., 0] = angle * 180 / np.pi / 2
        
    # Sets image value according to the optical flow 
    # magnitude (normalized) 
    mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)

    thresh_val = 7.5
    _, thresholded_mask = cv2.threshold(magnitude, thresh_val, 255, cv2.THRESH_BINARY)
    # thresholded_mask = mask[:,:,2]
    # thresholded_mask = ~thresholded_mask

    times.append(time.time() - start_time)
    ax[1].imshow(cv2.cvtColor(mask, cv2.COLOR_HSV2RGB))
    ax[2].imshow(thresholded_mask, cmap='gray')

    masks.append(thresholded_mask)
    plt.show()
    prev_gray = gray
    n += 1